In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy, IndependentlyCenter
from neuralnilm.data.datathread import DataThread

In [2]:
NILMTK_FILENAME = '/data/mine/vadeec/merged/ukdale.h5'
TARGET_APPLIANCE = 'kettle'
SEQ_LENGTH = 256
SAMPLE_PERIOD = 6
STRIDE = SEQ_LENGTH
WINDOWS = {
    'train': {
        1: ("2014-01-01", "2014-02-01")
    },
    'unseen_activations_of_seen_appliances': {
        1: ("2014-02-02", "2014-02-08")                
    },
    'unseen_appliances': {
        2: ("2013-06-01", "2013-06-07")
    }
}

LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename=NILMTK_FILENAME,
        sample_period=SAMPLE_PERIOD,
        windows=WINDOWS
    )
}



In [3]:
from neuralnilm.data.stridesource import StrideSource

In [4]:
stride_source = StrideSource(
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    filename=NILMTK_FILENAME,
    windows=WINDOWS,
    sample_period=SAMPLE_PERIOD,
    stride=STRIDE
)

INFO:neuralnilm.data.stridesource:Loading NILMTK data...
INFO:neuralnilm.data.stridesource:Loading data for UK-DALE_building_1...
INFO:neuralnilm.data.stridesource:Loaded data from building UK-DALE_building_1 for fold train from 2014-01-01 00:00:06+00:00 to 2014-01-31 23:59:54+00:00.
INFO:neuralnilm.data.stridesource:Loading data for UK-DALE_building_2...
INFO:neuralnilm.data.stridesource:Loaded data from building UK-DALE_building_2 for fold unseen_appliances from 2013-06-01 00:00:00+01:00 to 2013-06-06 23:59:54+01:00.
INFO:neuralnilm.data.stridesource:Loading data for UK-DALE_building_1...
INFO:neuralnilm.data.stridesource:Loaded data from building UK-DALE_building_1 for fold unseen_activations_of_seen_appliances from 2014-02-02 00:00:00+00:00 to 2014-02-07 23:59:54+00:00.
INFO:neuralnilm.data.stridesource:Done loading NILMTK mains data.


In [5]:
nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

INFO:neuralnilm.data.loadactivations:Loading NILMTK activations...
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 111 kettle activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 114 microwave activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 23 washing machine activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 kettle activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 microwave activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading washing mac

In [6]:
nilmtk_activations.keys()

['train', 'unseen_activations_of_seen_appliances', 'unseen_appliances']

In [7]:
from neuralnilm.data.realaggregatesource import RealAggregateSource

In [8]:
ras = RealAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    filename=NILMTK_FILENAME,
    windows=WINDOWS,
    sample_period=SAMPLE_PERIOD
)

INFO:neuralnilm.data.realaggregatesource:Loading NILMTK mains...
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold train from 2014-01-01 00:00:00+00:00 to 2014-01-31 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_2...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_2 for fold unseen_appliances from 2013-06-01 00:00:00+01:00 to 2013-06-06 23:59:54+01:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold unseen_activations_of_seen_appliances from 2014-02-02 00:00:00+00:00 to 2014-02-07 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Done loading NILMTK mains data.
INFO:neuralnilm.data.realaggregatesource:Found 89 sections without targ

In [9]:
ras.target_inclusion_prob = 0.5
for i in range(50):
    seq = ras.get_sequence()

In [10]:
#fig, axes = plt.subplots(2)
#axes[0].plot(seq.input)
#axes[1].plot(seq.target)
#plt.show()

In [11]:
nilmtk_activations['train']['kettle']['UK-DALE_building_1'][0]._metadata

['name', 'building', 'appliance', 'fold']

In [12]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    allow_incomplete_target=False,
    sample_period=SAMPLE_PERIOD
)

In [13]:
FOLD = 'train'
#FOLD = 'unseen_activations_of_seen_appliances'
#FOLD = 'unseen_appliances'
seq = source.get_sequence(enable_all_appliances=True)

In [14]:
#all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

In [15]:
#fig, axes = plt.subplots(2, sharex=True)
#seq.all_appliances.plot(ax=axes[0])
#axes[1].plot(seq.input)
#fig.suptitle(FOLD)
#plt.show()

In [16]:
sample = source.get_batch(num_seq_per_batch=1024).next()

In [17]:
pipeline = DataPipeline(
    sources=[source, ras, stride_source],
    num_seq_per_batch=64,
    input_processing=[DivideBy(sample.before_processing.input.flatten().std()), IndependentlyCenter()],
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

# Disaggregation

nilmtk_disag_source = NILMTKDisagSource(
    filename=NILMTK_FILENAME,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    buildings=[5],
    window_per_building={},
    stride=STRIDE,
    sample_period=SAMPLE_PERIOD
)

disag_pipeline = deepcopy(pipeline)
disag_pipeline.source = nilmtk_disag_source

disaggregator = Disaggregator(
    pipeline=disag_pipeline,
    output_path=PATH  # 
)

Disagregator ideas:

* make a copy of pipeline but swap source for a NILMTKDisagSource
* NILMTKDisagSource loads all data into memory (?) and iterates over chunks of it (get seq_length from pipeline.source.seq_length)
* 

In [19]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {
        'dense_layer_0': 100,
        'dense_layer_1':  50,
        'dense_layer_2': 100
    }

    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    
    # Dense layers
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    dense_layer_1 = DenseLayer(
        dense_layer_0, 
        num_units=NUM_UNITS['dense_layer_1']
    )
    dense_layer_2 = DenseLayer(
        dense_layer_1, 
        num_units=NUM_UNITS['dense_layer_2']
    )
    
    # Output
    final_dense_layer = DenseLayer(
        dense_layer_2,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [20]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [23]:
from neuralnilm.trainer import Trainer
from neuralnilm.metrics import Metrics

trainer = Trainer(
    net=net,
    data_pipeline=pipeline,
    experiment_id=["3"],
    metrics=Metrics(state_boundaries=[4]),
    requested_learning_rates={0: 1E-2},
    repeat_callbacks=[
        (1000, Trainer.save_params),
        (1000, Trainer.plot_estimates),
        ( 500, Trainer.validate)
    ]
)

Database already has an experiment with _id == 3. Should the old experiment be deleted (both from the database and from disk)? Or quit? [Q/d] d


INFO:neuralnilm.trainer:Deleting documents for old experiment.
INFO:neuralnilm.trainer:Directory exists = '/home/dk3810/temp/neural_nilm/output/3'
INFO:neuralnilm.trainer:  Deleting directory.


In [24]:
report = trainer.report()
report['data']['activations'] = LOADER_CONFIG
from neuralnilm.utils import sanitise_dict_for_mongo
sanitise_dict_for_mongo(report)

{'_id': '3',
 'data': {'activations': {'nilmtk_activations': {'appliances': ['kettle',
     'microwave',
     'washing machine'],
    'filename': '/data/mine/vadeec/merged/ukdale.h5',
    'sample_period': 6,
    'windows': {'train': {'1': ('2014-01-01', '2014-02-01')},
     'unseen_activations_of_seen_appliances': {'1': ('2014-02-02',
       '2014-02-08')},
     'unseen_appliances': {'2': ('2013-06-01', '2013-06-07')}}}},
  'pipeline': {'input_processing': [{'divisor': 630.8582153320312,
     'name': 'DivideBy'},
    {'name': 'IndependentlyCenter'}],
   'num_seq_per_batch': 64,
   'rng_seed': None,
   'source_probabilities': [0.3333333333333333,
    0.3333333333333333,
    0.3333333333333333],
   'sources': {'0': {'allow_incomplete_distractors': True,
     'allow_incomplete_target': False,
     'distractor_inclusion_prob': 0.25,
     'include_incomplete_target_in_output': True,
     'name': 'SyntheticAggregateSource',
     'num_batches_for_validation': 16,
     'rng_seed': None,
     '

In [ ]:
trainer.fit(50000)

INFO:neuralnilm.trainer:Starting training for 50000 iterations.
INFO:neuralnilm.trainer:Iteration 0: Change learning rate to 1.0E-02
INFO:neuralnilm.trainer:Compiling train cost function...
INFO:neuralnilm.trainer:Done compiling cost function.


   Update # |  Train cost  | Secs per update | Source ID
------------|--------------|-----------------|-----------
          0 |   1.429827  |    1.248209     |   1

INFO:neuralnilm.trainer:Iteration 0: Saving params.
INFO:neuralnilm.trainer:Done saving params.
INFO:neuralnilm.trainer:Iteration 0: Plotting estimates.
INFO:neuralnilm.net:Compiling deterministic output function...
INFO:neuralnilm.net:Done compiling deterministic output function.
INFO:neuralnilm.trainer:Done plotting.



          1 |   0.962788  |    0.167474     |   1
          2 |   1.147048  |    0.011550     |   0
          3 |   1.226706  |    0.047219     |   0
          4 |   0.161934  |    0.021605     |   2
          5 |   0.815449  |    0.142963     |   1
          6 |   1.271263  |    0.009346     |   0
          7 |   0.867317  |    0.138651     |   1
          8 |   1.111317  |    0.015524     |   0
          9 |   0.944497  |    0.095673     |   1
         10 |   0.184636  |    0.016405     |   2
         11 |   0.248021  |    0.040122     |   2
         12 |   1.035424  |    0.019694     |   0
         13 |   0.223985  |    0.029904     |   2
         14 |   0.814798  |    0.137157     |   1
         15 |   0.112299  |    0.007988     |   2
         16 |   1.006038  |    0.137333     |   1
         17 |   0.804864  |    0.007316     |   0
         18 |   1.058033  |    0.027261     |   0
         19 |   1.091661  |    0.031806     |   0
         20 |   0.235815  |    0.013456     |   2

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
INFO:neuralnilm.trainer:Iteration 89: Finished training epoch


...


      20077 |   0.047526  |    0.015998     |   2
      20078 |   0.161493  |    0.110576     |   1
      20079 |   0.262322  |    0.057404     |   1
      20080 |   0.255524  |    0.082414     |   1
      20081 |   0.051431  |    0.045988     |   2
      20082 |   0.174513  |    0.017818     |   0
      20083 |   0.035850  |    0.012200     |   2
      20084 |   0.173349  |    0.051880     |   0
      20085 |   0.181581  |    0.067144     |   1
      20086 |   0.205421  |    0.109247     |   1
      20087 |   0.209804  |    0.072955     |   1
      20088 |   0.222911  |    0.079097     |   1
      20089 |   0.188629  |    0.068583     |   1
      20090 |   0.226041  |    0.107516     |   1
      20091 |   0.197564  |    0.043514     |   1
      20092 |   0.037926  |    0.008927     |   2
      20093 |   0.045672  |    0.033902     |   2
      20094 |   0.248963  |    0.137890     |   1
      20095 |   0.028705  |    0.010642     |   2
      20096 |   0.177235  |    0.075082     |   1

INFO:neuralnilm.trainer:Iteration 20176: Finished training epoch



      20175 |   0.046896  |    0.012457     |   2
      20176 |   0.192155  |    0.046096     |   1
      20177 |   0.048215  |    0.035199     |   2
      20178 |   0.165777  |    0.089414     |   1
      20179 |   0.213301  |    0.014405     |   0
      20180 |   0.216193  |    0.136476     |   1
      20181 |   0.035940  |    0.007468     |   2
      20182 |   0.202484  |    0.034704     |   0
      20183 |   0.205099  |    0.064602     |   1
      20184 |   0.239531  |    0.044522     |   0
      20185 |   0.040515  |    0.008005     |   2
      20186 |   0.128130  |    0.022457     |   0
      20187 |   0.157219  |    0.147441     |   1
      20188 |   0.045749  |    0.012336     |   2
      20189 |   0.172912  |    0.084923     |   1
      20190 |   0.173204  |    0.093061     |   1
      20191 |   0.028371  |    0.013642     |   2
      20192 |   0.036582  |    0.046281     |   2
      20193 |   0.171980  |    0.015099     |   0
      20194 |   0.049702  |    0.037516     |   2

INFO:neuralnilm.trainer:Iteration 20268: Finished training epoch



      20268 |   0.190946  |    0.043746     |   0
      20269 |   0.181803  |    0.026680     |   0
      20270 |   0.273150  |    0.094663     |   1
      20271 |   0.244734  |    0.105353     |   1
      20272 |   0.050047  |    0.016564     |   2
      20273 |   0.037252  |    0.032833     |   2
      20274 |   0.129885  |    0.010452     |   0
      20275 |   0.039847  |    0.045829     |   2
      20276 |   0.164420  |    0.018364     |   0
      20277 |   0.250874  |    0.152828     |   1
      20278 |   0.135659  |    0.014748     |   0
      20279 |   0.246845  |    0.085833     |   1
      20280 |   0.227081  |    0.101589     |   1
      20281 |   0.236107  |    0.022723     |   0
      20282 |   0.168427  |    0.145199     |   1
      20283 |   0.224794  |    0.090901     |   1
      20284 |   0.047746  |    0.011712     |   2
      20285 |   0.155747  |    0.007305     |   0
      20286 |   0.266412  |    0.045511     |   0
      20287 |   0.029935  |    0.021653     |   2

In [ ]:
net.train_iterations